# pandas 库总体说明
Pandas 基于 NumPy、SciPy 补充了大量数据操作功能,能实现统计、分组、排序、透视表,可以代替 Excel 的绝大部分功能。

Pandas 主要有 2 种重要数据类型:Series、DataFrame(一维序列、二维表)。数据类型的转换需要用到 pd.Series/DataFrame.

## 1)Series

可以是一个样本的所有观测值或一组样本的某一属性的观测值。

如利用 NumPy 生成一个正态分布的随机数列,共含 4 个值。

Series1 = pd.Series(np.random.randn(4))

结果就自动添加了行索引 index。
0   -1.344609

1   0.177173

2   0.554958

3   -0.576237

过滤 Series 的方法是:print Series1 < 0 或 print Series1[Series1 < 0]。前者给出 Boolean 类
型的输出,后者给出具体的数值,仅仅输出 Series 中小于 0 的数值。

可以使用 Key-Value 的方式存储数据:

Series2 = pd.Series(Series1.values, index = ["row_" + unicode(i) for i in range(4)])

同样,Python 的基础数据结构字典也可以转化为 Series。

Series3 = pd.Series({"China": "Beijing", "England": "GB", "Japan": "Tokyo"})

输出结果依旧是一个序列, 但是因为字典本身是无序的, 所有有可能会打乱原字典的顺序。如果需要顺便不变,可以使用下面的方法明确指定这种秩序:

Series4_IndexList = ["China", "Japan", "England"]

Series4 = pd.Series(Series3, index = Series4_IndexList)

某些时候,Index 列表没有相应的对应值,这样会默认填补为空值,可以使用 isnull(0,notnull()来返回 Boolean 结果。

Series5_IndexList = ["A", "B", "C", "C"]

Series5 = pd.Series(Series1.values, index = Series5_IndexList)

index 允许重复,但是这样容易导致错误。

## 2)DataFrame

DataFrame 可以视作 Series 的有序集合, 可以从数据库、NumPy 二维数组、JSON 中定义数据框。

NumPy 二维数组:

DF1 = pd.DataFrame(np.asarray([("Japan", "Tokyo", 4000), ("S.Korea", "Seoul", 1000),("China", "Beijing", 9000)]),
columns = ["nation", "capital", "GDP"])

JSON:

DF2 = pd.DataFrame({"nation": ["Japan", "S.Korea", "China"], "capital": ["Tokyo", "Seoul","Beijing"], 
"GDP": [4000, 1000, 9000]})

但是字典的 key 是无序的,所以我们又要用到刚才 Series 中的类似方法加以解决:

DF3 = pd.DataFrame(DF2, columns = ["nation", "capital", "GDP"])

对应地,还可以人为指定行标秩序。

DF4 = pd.DataFrame(DF2, columns = ["nation", "capital", "GDP"], index = [2, 0, 1])

在 DataFrame 中切片:

取列:推荐使用 DF4["GDP"],最好别用 DF4.GDP,容易与一些关键字(保留字)冲突

取行:DF4[0: 1]或者 DF4.ix[0]

区别在于前者取了第一行,后者取了 index(行标)为 0 的第一行。

此外,如果要在数据框动态增加列,不能用.的方式,而要用[]

DF4["region"] = "East Asian"

# 代表性函数的使用介绍

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## 一、创建对象
1、可以通过传递一个 list 对象来创建一个 Series

In [2]:
s = pd.Series([1,3,5,np.nan,6,8])
print s

0    1.0
1    3.0
2    5.0
3    NaN
4    6.0
5    8.0
dtype: float64


2、通过传递一个 numpy array,时间索引以及列标签来创建一个 DataFrame:

In [3]:
dates = pd.date_range('20130101', periods = 6)
print dates

DatetimeIndex(['2013-01-01', '2013-01-02', '2013-01-03', '2013-01-04',
               '2013-01-05', '2013-01-06'],
              dtype='datetime64[ns]', freq='D')


In [4]:
df = pd.DataFrame(np.random.randn(6,4), index = dates, columns = list('ABCD'))
print df

                   A         B         C         D
2013-01-01  0.026535 -0.907195  0.249514 -1.079940
2013-01-02 -1.252509 -0.664995 -0.261090 -0.815005
2013-01-03  1.380034 -0.118999 -0.897727  0.185313
2013-01-04 -0.514970 -0.187767 -0.443209  1.035802
2013-01-05 -1.086688  1.220295 -0.618035  0.639486
2013-01-06  0.615427  0.510758 -0.285427 -1.277933


3、通过传递一个能够被转换成类似序列结构的字典对象来创建一个 DataFrame:

In [5]:
df2 = pd.DataFrame({ 'A' : 1.,
....:                'B' : pd.Timestamp('20130102'),
....:                'C' : pd.Series(1,index=list(range(4)),dtype='float32'),
....:                'D' : np.array([3] * 4,dtype='int32'),
....:                'E' : pd.Categorical(["test","train","test","train"]),
....:                'F' : 'foo' })

print df2

     A          B    C  D      E    F
0  1.0 2013-01-02  1.0  3   test  foo
1  1.0 2013-01-02  1.0  3  train  foo
2  1.0 2013-01-02  1.0  3   test  foo
3  1.0 2013-01-02  1.0  3  train  foo


4、查看不同列的数据类型:

In [6]:
print df2.dtypes

A           float64
B    datetime64[ns]
C           float32
D             int32
E          category
F            object
dtype: object


## 二、查看数据
1、 查看 frame 中头部和尾部的行:

In [7]:
print df.head()

                   A         B         C         D
2013-01-01  0.026535 -0.907195  0.249514 -1.079940
2013-01-02 -1.252509 -0.664995 -0.261090 -0.815005
2013-01-03  1.380034 -0.118999 -0.897727  0.185313
2013-01-04 -0.514970 -0.187767 -0.443209  1.035802
2013-01-05 -1.086688  1.220295 -0.618035  0.639486


In [8]:
print df.tail(3)

                   A         B         C         D
2013-01-04 -0.514970 -0.187767 -0.443209  1.035802
2013-01-05 -1.086688  1.220295 -0.618035  0.639486
2013-01-06  0.615427  0.510758 -0.285427 -1.277933


2、 显示索引、列和底层的 numpy 数据:

In [9]:
print df.index

DatetimeIndex(['2013-01-01', '2013-01-02', '2013-01-03', '2013-01-04',
               '2013-01-05', '2013-01-06'],
              dtype='datetime64[ns]', freq='D')


In [10]:
print df.columns

Index([u'A', u'B', u'C', u'D'], dtype='object')


In [11]:
print df.values

[[ 0.02653459 -0.90719464  0.24951423 -1.07994048]
 [-1.25250851 -0.66499466 -0.26108965 -0.81500487]
 [ 1.38003356 -0.11899932 -0.89772703  0.18531335]
 [-0.51497049 -0.18776652 -0.44320899  1.03580206]
 [-1.08668828  1.2202953  -0.61803527  0.63948616]
 [ 0.61542686  0.51075821 -0.28542684 -1.27793326]]


3、 describe()函数对于数据的快速统计汇总:

In [12]:
print df.describe()

              A         B         C         D
count  6.000000  6.000000  6.000000  6.000000
mean  -0.138695 -0.024650 -0.375996 -0.218713
std    1.018174  0.782430  0.386775  0.968791
min   -1.252509 -0.907195 -0.897727 -1.277933
25%   -0.943759 -0.545688 -0.574329 -1.013707
50%   -0.244218 -0.153383 -0.364318 -0.314846
75%    0.468204  0.353319 -0.267174  0.525943
max    1.380034  1.220295  0.249514  1.035802


4、 对数据的转置:

In [13]:
print df.T

   2013-01-01  2013-01-02  2013-01-03  2013-01-04  2013-01-05  2013-01-06
A    0.026535   -1.252509    1.380034   -0.514970   -1.086688    0.615427
B   -0.907195   -0.664995   -0.118999   -0.187767    1.220295    0.510758
C    0.249514   -0.261090   -0.897727   -0.443209   -0.618035   -0.285427
D   -1.079940   -0.815005    0.185313    1.035802    0.639486   -1.277933


5、 按轴进行排序

In [14]:
print df.sort_index(axis = 1, ascending = False)

                   D         C         B         A
2013-01-01 -1.079940  0.249514 -0.907195  0.026535
2013-01-02 -0.815005 -0.261090 -0.664995 -1.252509
2013-01-03  0.185313 -0.897727 -0.118999  1.380034
2013-01-04  1.035802 -0.443209 -0.187767 -0.514970
2013-01-05  0.639486 -0.618035  1.220295 -1.086688
2013-01-06 -1.277933 -0.285427  0.510758  0.615427


6、 按值进行排序

In [15]:
print df.sort_values(by='B')

                   A         B         C         D
2013-01-01  0.026535 -0.907195  0.249514 -1.079940
2013-01-02 -1.252509 -0.664995 -0.261090 -0.815005
2013-01-04 -0.514970 -0.187767 -0.443209  1.035802
2013-01-03  1.380034 -0.118999 -0.897727  0.185313
2013-01-06  0.615427  0.510758 -0.285427 -1.277933
2013-01-05 -1.086688  1.220295 -0.618035  0.639486


## 三、选择
虽然标准的 Python/Numpy 的选择和设置表达式都能够直接派上用场,但是作为工程
使用的代码,推荐使用经过优化的 pandas 数据访问方式: .at, .iat, .loc, .iloc 和 .ix。

### 获取
1、 选择一个单独的列,这将会返回一个 Series,等同于 df.A:

In [16]:
print df['A']

2013-01-01    0.026535
2013-01-02   -1.252509
2013-01-03    1.380034
2013-01-04   -0.514970
2013-01-05   -1.086688
2013-01-06    0.615427
Freq: D, Name: A, dtype: float64


2、 通过[]进行选择,这将会对行进行切片

In [17]:
print df[0:3]

                   A         B         C         D
2013-01-01  0.026535 -0.907195  0.249514 -1.079940
2013-01-02 -1.252509 -0.664995 -0.261090 -0.815005
2013-01-03  1.380034 -0.118999 -0.897727  0.185313


In [18]:
print df['20130102':'20130104']

                   A         B         C         D
2013-01-02 -1.252509 -0.664995 -0.261090 -0.815005
2013-01-03  1.380034 -0.118999 -0.897727  0.185313
2013-01-04 -0.514970 -0.187767 -0.443209  1.035802


### 通过标签选择
1、 使用标签来获取一个交叉的区域

In [19]:
print df.loc[dates[0]]

A    0.026535
B   -0.907195
C    0.249514
D   -1.079940
Name: 2013-01-01 00:00:00, dtype: float64


2、 通过标签来在多个轴上进行选择

In [20]:
print df.loc[:,['A','B']]

                   A         B
2013-01-01  0.026535 -0.907195
2013-01-02 -1.252509 -0.664995
2013-01-03  1.380034 -0.118999
2013-01-04 -0.514970 -0.187767
2013-01-05 -1.086688  1.220295
2013-01-06  0.615427  0.510758


3、 标签切片

In [21]:
print df.loc['20130102':'20130104',['A','B']]

                   A         B
2013-01-02 -1.252509 -0.664995
2013-01-03  1.380034 -0.118999
2013-01-04 -0.514970 -0.187767


4、 对于返回的对象进行维度缩减

In [22]:
print df.loc['20130102',['A','B']]

A   -1.252509
B   -0.664995
Name: 2013-01-02 00:00:00, dtype: float64


5、 获取一个标量

In [23]:
print df.loc[dates[0],'A']

0.0265345851963


6、 快速访问一个标量(与上一个方法等价)

In [24]:
print df.at[dates[0],'A']

0.0265345851963


### 通过位置选择
1、 通过传递数值进行位置选择(选择的是行)

In [25]:
print df.iloc[3]

A   -0.514970
B   -0.187767
C   -0.443209
D    1.035802
Name: 2013-01-04 00:00:00, dtype: float64


2、 通过数值进行切片,与 numpy/python 中的情况类似

In [26]:
print df.iloc[3:5,0:2]

                   A         B
2013-01-04 -0.514970 -0.187767
2013-01-05 -1.086688  1.220295


3、 通过指定一个位置的列表,与 numpy/python 中的情况类似

In [27]:
print df.iloc[[1,2,4],[0,2]]

                   A         C
2013-01-02 -1.252509 -0.261090
2013-01-03  1.380034 -0.897727
2013-01-05 -1.086688 -0.618035


4、对行进行切片

In [28]:
print df.iloc[:,1:3]

                   B         C
2013-01-01 -0.907195  0.249514
2013-01-02 -0.664995 -0.261090
2013-01-03 -0.118999 -0.897727
2013-01-04 -0.187767 -0.443209
2013-01-05  1.220295 -0.618035
2013-01-06  0.510758 -0.285427


5、 对列进行切片

In [29]:
print df.iloc[:,1:3]

                   B         C
2013-01-01 -0.907195  0.249514
2013-01-02 -0.664995 -0.261090
2013-01-03 -0.118999 -0.897727
2013-01-04 -0.187767 -0.443209
2013-01-05  1.220295 -0.618035
2013-01-06  0.510758 -0.285427


6、 获取特定的值

In [30]:
print df.iloc[1,1]

-0.664994663668


In [31]:
print df.iat[1,1]

-0.664994663668


### 布尔索引
1、 使用一个单独列的值来选择数据:

In [32]:
print df[df.A > 0]

                   A         B         C         D
2013-01-01  0.026535 -0.907195  0.249514 -1.079940
2013-01-03  1.380034 -0.118999 -0.897727  0.185313
2013-01-06  0.615427  0.510758 -0.285427 -1.277933


2、选择数据:

In [33]:
print df[df > 0]

                   A         B         C         D
2013-01-01  0.026535       NaN  0.249514       NaN
2013-01-02       NaN       NaN       NaN       NaN
2013-01-03  1.380034       NaN       NaN  0.185313
2013-01-04       NaN       NaN       NaN  1.035802
2013-01-05       NaN  1.220295       NaN  0.639486
2013-01-06  0.615427  0.510758       NaN       NaN


3、 使用 isin()方法来过滤:

In [34]:
df2 = df.copy()
df2['E'] = ['one', 'one','two','three','four','three']
print df2

                   A         B         C         D      E
2013-01-01  0.026535 -0.907195  0.249514 -1.079940    one
2013-01-02 -1.252509 -0.664995 -0.261090 -0.815005    one
2013-01-03  1.380034 -0.118999 -0.897727  0.185313    two
2013-01-04 -0.514970 -0.187767 -0.443209  1.035802  three
2013-01-05 -1.086688  1.220295 -0.618035  0.639486   four
2013-01-06  0.615427  0.510758 -0.285427 -1.277933  three


In [35]:
print df2[df2['E'].isin(['two','four'])]

                   A         B         C         D     E
2013-01-03  1.380034 -0.118999 -0.897727  0.185313   two
2013-01-05 -1.086688  1.220295 -0.618035  0.639486  four


### 设置
1、 设置一个新的列:

In [36]:
s1 = pd.Series([1,2,3,4,5,6], index = pd.date_range('20130102', periods = 6))
print s1

2013-01-02    1
2013-01-03    2
2013-01-04    3
2013-01-05    4
2013-01-06    5
2013-01-07    6
Freq: D, dtype: int64


In [37]:
df['F'] = s1
print df

                   A         B         C         D    F
2013-01-01  0.026535 -0.907195  0.249514 -1.079940  NaN
2013-01-02 -1.252509 -0.664995 -0.261090 -0.815005  1.0
2013-01-03  1.380034 -0.118999 -0.897727  0.185313  2.0
2013-01-04 -0.514970 -0.187767 -0.443209  1.035802  3.0
2013-01-05 -1.086688  1.220295 -0.618035  0.639486  4.0
2013-01-06  0.615427  0.510758 -0.285427 -1.277933  5.0


2、 通过标签设置新的值:

In [38]:
df.at[dates[0],'A'] = 0

3、 通过位置设置新的值:

In [39]:
df.iat[0,1] = 0

4、 通过一个 numpy 数组设置一组新值:

In [40]:
df.loc[:,'D'] = np.array([5] * len(df))

In [41]:
print df

                   A         B         C  D    F
2013-01-01  0.000000  0.000000  0.249514  5  NaN
2013-01-02 -1.252509 -0.664995 -0.261090  5  1.0
2013-01-03  1.380034 -0.118999 -0.897727  5  2.0
2013-01-04 -0.514970 -0.187767 -0.443209  5  3.0
2013-01-05 -1.086688  1.220295 -0.618035  5  4.0
2013-01-06  0.615427  0.510758 -0.285427  5  5.0


5、 通过 where 操作来设置新的值:

In [42]:
df2 = df.copy()
df2[df2 > 0] = -df2
print df2

                   A         B         C  D    F
2013-01-01  0.000000  0.000000 -0.249514 -5  NaN
2013-01-02 -1.252509 -0.664995 -0.261090 -5 -1.0
2013-01-03 -1.380034 -0.118999 -0.897727 -5 -2.0
2013-01-04 -0.514970 -0.187767 -0.443209 -5 -3.0
2013-01-05 -1.086688 -1.220295 -0.618035 -5 -4.0
2013-01-06 -0.615427 -0.510758 -0.285427 -5 -5.0


## 四、缺失值处理
在 pandas 中,使用 np.nan 来代替缺失值,这些值将默认不会包含在计算中

1、reindex()方法可以对指定轴上的索引进行改变/增加/删除操作,这将返回原始数据
的一个拷贝:

In [43]:
df1 = df.reindex(index = dates[0:4], columns = list(df.columns) + ['E'])
df1.loc[dates[0]:dates[1],'E'] = 1
print df1

                   A         B         C  D    F    E
2013-01-01  0.000000  0.000000  0.249514  5  NaN  1.0
2013-01-02 -1.252509 -0.664995 -0.261090  5  1.0  1.0
2013-01-03  1.380034 -0.118999 -0.897727  5  2.0  NaN
2013-01-04 -0.514970 -0.187767 -0.443209  5  3.0  NaN


2、 去掉包含缺失值的行:

In [44]:
print df1.dropna(how = 'any')

                   A         B        C  D    F    E
2013-01-02 -1.252509 -0.664995 -0.26109  5  1.0  1.0


3、 对缺失值进行填充:

In [45]:
print df1.fillna(value = 5)

                   A         B         C  D    F    E
2013-01-01  0.000000  0.000000  0.249514  5  5.0  1.0
2013-01-02 -1.252509 -0.664995 -0.261090  5  1.0  1.0
2013-01-03  1.380034 -0.118999 -0.897727  5  2.0  5.0
2013-01-04 -0.514970 -0.187767 -0.443209  5  3.0  5.0


4、 对数据进行布尔填充:

In [46]:
print pd.isnull(df1)

                A      B      C      D      F      E
2013-01-01  False  False  False  False   True  False
2013-01-02  False  False  False  False  False  False
2013-01-03  False  False  False  False  False   True
2013-01-04  False  False  False  False  False   True


##  5、操作
统计(相关操作通常情况下不包括缺失值)

1、 执行描述性统计:

In [47]:
print df.mean()

A   -0.143118
B    0.126549
C   -0.375996
D    5.000000
F    3.000000
dtype: float64


2、 在其他轴上进行相同的操作:

In [48]:
print df.mean(1)

2013-01-01    1.312379
2013-01-02    0.764281
2013-01-03    1.472661
2013-01-04    1.370811
2013-01-05    1.703114
2013-01-06    2.168152
Freq: D, dtype: float64


3、 对于拥有不同维度,需要对齐的对象进行操作。Pandas 会自动的沿着指定的维
度进行广播:

In [49]:
s = pd.Series([1,3,5,np.nan,6,8], index = dates).shift(2)
print s

2013-01-01    NaN
2013-01-02    NaN
2013-01-03    1.0
2013-01-04    3.0
2013-01-05    5.0
2013-01-06    NaN
Freq: D, dtype: float64


In [50]:
print df.sub(s, axis = 'index')

                   A         B         C    D    F
2013-01-01       NaN       NaN       NaN  NaN  NaN
2013-01-02       NaN       NaN       NaN  NaN  NaN
2013-01-03  0.380034 -1.118999 -1.897727  4.0  1.0
2013-01-04 -3.514970 -3.187767 -3.443209  2.0  0.0
2013-01-05 -6.086688 -3.779705 -5.618035  0.0 -1.0
2013-01-06       NaN       NaN       NaN  NaN  NaN


### Apply
1、 对数据应用函数:

In [51]:
print df.apply(np.cumsum)

                   A         B         C   D     F
2013-01-01  0.000000  0.000000  0.249514   5   NaN
2013-01-02 -1.252509 -0.664995 -0.011575  10   1.0
2013-01-03  0.127525 -0.783994 -0.909302  15   3.0
2013-01-04 -0.387445 -0.971761 -1.352511  20   6.0
2013-01-05 -1.474134  0.248535 -1.970547  25  10.0
2013-01-06 -0.858707  0.759293 -2.255974  30  15.0


In [52]:
print df.apply(lambda x: x.max() - x.min())

A    2.632542
B    1.885290
C    1.147241
D    0.000000
F    4.000000
dtype: float64


### 直方图

In [53]:
s = pd.Series(np.random.randint(0, 7, size = 10))
print s

0    5
1    1
2    6
3    3
4    4
5    6
6    3
7    3
8    4
9    4
dtype: int64


In [54]:
print s.value_counts()

4    3
3    3
6    2
5    1
1    1
dtype: int64


### 字符串方法

Series 对象在其 str 属性中配备了一组字符串处理方法,可以很容易的应用到数组中的
每个元素,如下段代码所示。变成小写。

In [55]:
s = pd.Series(['A', 'B', 'C', 'Aaba', 'Baca', np.nan, 'CABA', 'dog', 'cat'])
print s

0       A
1       B
2       C
3    Aaba
4    Baca
5     NaN
6    CABA
7     dog
8     cat
dtype: object


## 六、合并
Pandas 提供了大量的方法能够轻松的对 Series, DataFrame 和 Panel 对象进行各种符合各种逻辑关系的合并操作。

用 concat()把 pandas 类合并到一起:

In [56]:
df = pd.DataFrame(np.random.randn(10, 4))
print df

          0         1         2         3
0  1.041151 -0.526343  0.240190 -2.129342
1 -1.040013 -0.580216  0.054243 -0.496978
2 -0.829497  0.407056  0.888406 -1.072323
3  0.212972  0.224848 -1.027770  0.541093
4  0.357309 -0.303670 -2.614913  0.737872
5  1.187109 -0.664920 -2.769162 -0.245307
6 -1.000689 -1.158225 -0.715559 -0.291499
7  0.656899 -0.200471 -1.039671 -0.247050
8 -0.633796  0.461215  0.026049 -0.773767
9  1.888449 -0.293835 -0.617921  0.076162


In [57]:
pieces = [df[:3], df[3:7], df[7:]]

In [59]:
print pd.concat(pieces)

          0         1         2         3
0  1.041151 -0.526343  0.240190 -2.129342
1 -1.040013 -0.580216  0.054243 -0.496978
2 -0.829497  0.407056  0.888406 -1.072323
3  0.212972  0.224848 -1.027770  0.541093
4  0.357309 -0.303670 -2.614913  0.737872
5  1.187109 -0.664920 -2.769162 -0.245307
6 -1.000689 -1.158225 -0.715559 -0.291499
7  0.656899 -0.200471 -1.039671 -0.247050
8 -0.633796  0.461215  0.026049 -0.773767
9  1.888449 -0.293835 -0.617921  0.076162


### Join
Join 类似于 SQL 类型的合并

In [60]:
left = pd.DataFrame({'key': ['foo', 'foo'], 'lval': [1, 2]})
right = pd.DataFrame({'key': ['foo', 'foo'], 'rval': [4, 5]})

In [61]:
print left

   key  lval
0  foo     1
1  foo     2


In [62]:
print right

   key  rval
0  foo     4
1  foo     5


In [63]:
print pd.merge(left, right, on = 'key')

   key  lval  rval
0  foo     1     4
1  foo     1     5
2  foo     2     4
3  foo     2     5


### Append
Append 将一行连接到一个 DataFrame 上

In [64]:
df = pd.DataFrame(np.random.randn(8, 4), columns = ['A','B','C','D'])

In [65]:
print df

          A         B         C         D
0  2.399855 -0.610849 -0.671329  0.099010
1 -0.268582 -0.067634 -0.804054 -0.590644
2  1.639320  1.116574  0.471779 -1.594703
3 -0.354838 -0.466890 -0.161267  1.641098
4 -1.849251  1.352512  2.871678 -1.547155
5  1.126047  2.218345 -1.707503 -0.692559
6  0.737147  0.452606 -0.258848 -0.157267
7 -0.097699  0.965176  0.072476  0.054900


In [66]:
s = df.iloc[3]

In [67]:
print df.append(s, ignore_index = True)

          A         B         C         D
0  2.399855 -0.610849 -0.671329  0.099010
1 -0.268582 -0.067634 -0.804054 -0.590644
2  1.639320  1.116574  0.471779 -1.594703
3 -0.354838 -0.466890 -0.161267  1.641098
4 -1.849251  1.352512  2.871678 -1.547155
5  1.126047  2.218345 -1.707503 -0.692559
6  0.737147  0.452606 -0.258848 -0.157267
7 -0.097699  0.965176  0.072476  0.054900
8 -0.354838 -0.466890 -0.161267  1.641098


## 七、分组
对于”group by”操作,我们通常是指以下一个或多个操作步骤:

(Splitting)按照一些规则将数据分为不同的组;

(Applying)对于每组数据分别执行一个函数;

(Combining)将结果组合到一个数据结构中;

In [69]:
df = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar',
....:                     'foo', 'bar', 'foo', 'foo'],
....:              'B' : ['one', 'one', 'two', 'three',
....:                     'two', 'two', 'one', 'three'],
....:              'C' : np.random.randn(8),
....:              'D' : np.random.randn(8)})

In [70]:
print df

     A      B         C         D
0  foo    one -0.114526  1.375341
1  bar    one -1.563610 -1.371752
2  foo    two -0.131135 -2.044052
3  bar  three -1.049120 -0.774717
4  foo    two -0.137321 -1.438457
5  bar    two  1.322936 -0.654709
6  foo    one -0.351001 -0.457565
7  foo  three -0.824124  0.622835


1、 分组并对每个分组执行 sum 函数:

In [71]:
print df.groupby('A').sum()

            C         D
A                      
bar -1.289794 -2.801178
foo -1.558108 -1.941899


2、 通过多个列进行分组形成一个层次索引,然后执行函数:

In [72]:
print df.groupby(['A','B']).sum()

                  C         D
A   B                        
bar one   -1.563610 -1.371752
    three -1.049120 -0.774717
    two    1.322936 -0.654709
foo one   -0.465527  0.917776
    three -0.824124  0.622835
    two   -0.268456 -3.482509


### Reshaping
### Stack

In [73]:
tuples = list(zip(*[['bar', 'bar', 'baz', 'baz',
....:                'foo', 'foo', 'qux', 'qux'],
....:               ['one', 'two', 'one', 'two',
....:                'one', 'two', 'one', 'two']]))

In [75]:
index = pd.MultiIndex.from_tuples(tuples, names = ['first', 'second'])
df = pd.DataFrame(np.random.randn(8, 2), index = index, columns = ['A', 'B'])
df2 = df[:4]

In [76]:
print df2

                     A         B
first second                    
bar   one    -1.130398  0.862571
      two     1.046531 -1.895874
baz   one     0.278263 -0.278354
      two    -1.118441  1.408591


In [77]:
stacked = df2.stack()

In [78]:
print stacked

first  second   
bar    one     A   -1.130398
               B    0.862571
       two     A    1.046531
               B   -1.895874
baz    one     A    0.278263
               B   -0.278354
       two     A   -1.118441
               B    1.408591
dtype: float64


In [79]:
print stacked.unstack()

                     A         B
first second                    
bar   one    -1.130398  0.862571
      two     1.046531 -1.895874
baz   one     0.278263 -0.278354
      two    -1.118441  1.408591


In [80]:
print stacked.unstack(1)

second        one       two
first                      
bar   A -1.130398  1.046531
      B  0.862571 -1.895874
baz   A  0.278263 -1.118441
      B -0.278354  1.408591


In [81]:
print stacked.unstack(0)

first          bar       baz
second                      
one    A -1.130398  0.278263
       B  0.862571 -0.278354
two    A  1.046531 -1.118441
       B -1.895874  1.408591


### 数据透视表

In [82]:
df = pd.DataFrame({'A' : ['one', 'one', 'two', 'three'] * 3,
.....:             'B' : ['A', 'B', 'C'] * 4,
.....:             'C' : ['foo', 'foo', 'foo', 'bar', 'bar', 'bar'] * 2,
.....:             'D' : np.random.randn(12),
.....:             'E' : np.random.randn(12)})

In [83]:
print df

        A  B    C         D         E
0     one  A  foo -1.646620 -0.526991
1     one  B  foo  1.303054  1.560730
2     two  C  foo -2.288601 -0.568366
3   three  A  bar  1.132503  0.066476
4     one  B  bar -0.072598 -0.571294
5     one  C  bar -0.841017 -1.446070
6     two  A  foo  0.595537 -0.689051
7   three  B  foo -0.532413 -0.752946
8     one  C  foo  2.169197  0.951718
9     one  A  bar -0.448969  0.674647
10    two  B  bar  0.807108  1.418564
11  three  C  bar -2.802334 -0.490561


可以从这个数据中轻松的生成数据透视表:

In [84]:
print pd.pivot_table(df, values = 'D', index = ['A', 'B'], columns = ['C'])

C             bar       foo
A     B                    
one   A -0.448969 -1.646620
      B -0.072598  1.303054
      C -0.841017  2.169197
three A  1.132503       NaN
      B       NaN -0.532413
      C -2.802334       NaN
two   A       NaN  0.595537
      B  0.807108       NaN
      C       NaN -2.288601


## 九、导入和保存数据
CSV

1 、 写入 csv 文件:

In [85]:
df.to_csv('foo.csv')

In [86]:
print pd.read_csv('foo.csv')

    Unnamed: 0      A  B    C         D         E
0            0    one  A  foo -1.646620 -0.526991
1            1    one  B  foo  1.303054  1.560730
2            2    two  C  foo -2.288601 -0.568366
3            3  three  A  bar  1.132503  0.066476
4            4    one  B  bar -0.072598 -0.571294
5            5    one  C  bar -0.841017 -1.446070
6            6    two  A  foo  0.595537 -0.689051
7            7  three  B  foo -0.532413 -0.752946
8            8    one  C  foo  2.169197  0.951718
9            9    one  A  bar -0.448969  0.674647
10          10    two  B  bar  0.807108  1.418564
11          11  three  C  bar -2.802334 -0.490561


### HDF5
1、 写入 HDF5 存储:

In [87]:
df.to_hdf('foo.h5','df')

2、 从 HDF5 存储中读取:

In [88]:
print pd.read_hdf('foo.h5','df')

        A  B    C         D         E
0     one  A  foo -1.646620 -0.526991
1     one  B  foo  1.303054  1.560730
2     two  C  foo -2.288601 -0.568366
3   three  A  bar  1.132503  0.066476
4     one  B  bar -0.072598 -0.571294
5     one  C  bar -0.841017 -1.446070
6     two  A  foo  0.595537 -0.689051
7   three  B  foo -0.532413 -0.752946
8     one  C  foo  2.169197  0.951718
9     one  A  bar -0.448969  0.674647
10    two  B  bar  0.807108  1.418564
11  three  C  bar -2.802334 -0.490561


### Excel
1、 写入 excel 文件:

In [89]:
df.to_excel('foo.xlsx', sheet_name='Sheet1')

2、 从 excel 文件中读取:

In [90]:
print pd.read_excel('foo.xlsx', 'Sheet1', index_col = None, na_values = ['NA'])

        A  B    C         D         E
0     one  A  foo -1.646620 -0.526991
1     one  B  foo  1.303054  1.560730
2     two  C  foo -2.288601 -0.568366
3   three  A  bar  1.132503  0.066476
4     one  B  bar -0.072598 -0.571294
5     one  C  bar -0.841017 -1.446070
6     two  A  foo  0.595537 -0.689051
7   three  B  foo -0.532413 -0.752946
8     one  C  foo  2.169197  0.951718
9     one  A  bar -0.448969  0.674647
10    two  B  bar  0.807108  1.418564
11  three  C  bar -2.802334 -0.490561
